# Introduction

본래 목적은 아래 링크에서 작성한 MNIST GAN - Pytorch version을 Keras로 직접 옮겨보는 것이었습니다.

https://github.com/Jooong/Deep-Learning-Study/blob/master/2017-07-29-GAN-tutorial-2-MNIST.markdown

하지만 주먹구구식으로 코드를 작성한 결과,아래의 이미지에서 더이상 학습이 되지 않는 현상이 발생했습니다..
<img src = "https://scontent-hkg3-2.xx.fbcdn.net/v/t1.0-9/20479575_1545344795530358_5123431362607095007_n.jpg?oh=8f81a0bec8575f612918afa1239d88a4&oe=5A017A15"></img>

문제를 해결하기위해 갖은 노력을 해보았지만 in vain...

결국 "참고"만 하려던 아래 링크의 코드를 거의 갖다 쓰다시피 하게 되었습니다..

https://github.com/Zackory/Keras-MNIST-GAN/blob/master/mnist_gan.py

## Import needed modules

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import cufflinks as cf
cf.go_offline()

from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Reshape, Dense, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import backend as K
from keras import initializers

K.set_image_dim_ordering('th')

## Initialize some values & load MNIST data

In [ ]:
# Dim of noise vector == 100 -> most of GAN implementations do so.
noiseDim = 100
examples = 16

# set sample noise to keep track of how well Generator is trained
sample_noise = np.random.normal(0, 1, size=[examples, noiseDim])



# Load MNIST data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5)/127.5
X_train = X_train.reshape(60000, 784)
np.random.shuffle(X_train)

# make mini batches
BATCH_SIZE = 100
NUM_BATCH = X_train.shape[0] // BATCH_SIZE
X_batches = np.array([X_train[i:i+BATCH_SIZE] for i in range(NUM_BATCH)])

# List for tracking losses
dLosses = []
gLosses = []

## Set Optimizer

In [ ]:
# Optimizer
adam = Adam(lr=0.0002, beta_1=0.5)

## Build GAN

#### 1) Build Generator

In [ ]:
generator = Sequential()
# Layer1
generator.add(Dense(256, input_dim=noiseDim, init=initializers.RandomUniform(minval=-1, maxval=1, seed=None)))
generator.add(LeakyReLU(0.2))

# Layer2
generator.add(Dense(512))
generator.add(LeakyReLU(0.2))

# Layer3
generator.add(Dense(1024))
generator.add(LeakyReLU(0.2))

# Layer4
generator.add(Dense(784, activation='tanh'))
generator.compile(loss='binary_crossentropy', optimizer=adam)

#### 2) Build Discriminator

In [ ]:
discriminator = Sequential()
# Layer1
discriminator.add(Dense(1024, input_dim=784, init=initializers.RandomUniform(minval=-1, maxval=1, seed=None)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))

# Layer2
discriminator.add(Dense(512))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))

# Layer3
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))

# Layer4
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer=adam)

#### 3) Combine D & G

In [ ]:
discriminator.trainable = False    # disable training of discriminator while training Generator
ganInput = Input(shape=(noiseDim,)) # Init Keras Tensor
x = generator(ganInput)             # Generate Fake Images
ganOutput = discriminator(x)        # Test Discriminator with the Fake Images
gan = Model(input=ganInput, output=ganOutput)  # random noise -> GAN -> probability vector
gan.compile(loss='binary_crossentropy', optimizer=adam)

## Auxiliary Functions

In [ ]:
# Plot the loss from each batch
def plotLoss(epoch):
    df = pd.DataFrame([dLosses,gLosses]).T.rename(columns={0:"d_loss",1:"g_loss"})
    df.iplot(title="Tracking Losses of Discriminator and Generator",
             xTitle="Epoch",yTitle="Loss",
             filename='results/gan_loss_epoch_%d.png' % epoch)
    
    
# Create a wall of generated MNIST images
def plotGeneratedImages(epoch, noise, examples=16, dim=(4, 4), figsize=(10, 10)):

    generatedImages = generator.predict(noise)
    generatedImages = generatedImages.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(generatedImages.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generatedImages[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('results/gan_generated_image_epoch_%d.png' % epoch)

## Training Function

In [ ]:
def train(epochs=200, batchSize=100):
    batchCount = X_train.shape[0] // batchSize
    print( 'Epochs:', epochs)
    print( 'Batch size:', batchSize)
    print( 'Batches per epoch:', batchCount)

    for e in range(1, epochs+1):
        print( '-'*15, 'Epoch %d' % e, '-'*15)
        for i in tqdm_notebook(range(len(X_batches))):
            
            # Get a random set of input noise and images
            noise = np.random.normal(0, 1, size=[batchSize, randomDim])
            imageBatch = X_batches[i]
            

            # Generate fake MNIST images
            generatedImages = generator.predict(noise)
            
            # input Matrix for Discriminator
            X = np.concatenate([imageBatch, generatedImages])

            # Labels for generated and real data
            yLabel = np.zeros(2*batchSize)
            
            # Labels for real data == 1
            yLabel[:batchSize] = 1

            # Train discriminator
            discriminator.trainable = True
            dloss = discriminator.train_on_batch(X, yLabel)

            # Train generator
            noise = np.random.normal(0, 1, size=[batchSize, randomDim])
            yGen = np.ones(batchSize)
            discriminator.trainable = False
            gloss = gan.train_on_batch(noise, yGen)

        # Store loss of most recent batch from this epoch
        dLosses.append(dloss)
        gLosses.append(gloss)

        
        plotGeneratedImages(e, sample_noise)


        # Plot losses from every epoch
        plotLoss(e)

In [ ]:
train(200, 100)

---

# Pytorch v.s. Keras

### 1. Pytorch

* 모델을 Class로 구현해야한다.(Pytorch 자체가 그렇게 강제한다)
    * init에서 레이어 빌딩 등 모델에 대한 그래프를 그려야 하고
    * forward()메소드를 구현하여 모델의 output을 명시해야한다.
    * 레이어를 쌓는 방법은 간단하다. Sequential() 객체 안에 레이어를 인자로 순서대로 넣어주면 된다.
    
    
* Backpropagation과 parameter update를 명시해야 한다.
    * model 변수에 .back()과 .step() 메소드를 사용하면 된다.
    * GAN처럼 서로 다른 뉴럴 네트워크가 한 모델 안에 존재하는 경우, 각각의 네트워크를 따로 학습시키기가 쉽다.
        * back()과 step()을 명시하므로써 뭔가 내가 학습과정을 더 쉽게 control할 수 있는 것 같다.
        * 결정적으로 Pytorch가 Keras보다 코드를 짜다고 쉽게 느낀 부분은, D와 G 두 네트워크를 별개로 학습시킬 수 있었기 때문이다.
        * Keras는 generator와 discriminator를 stack한 GAN 모델 자체를 명시해야 하는데, 이 부분이 그렇게 직관적으로 이해되지는 않았다.
        

* Mini Batch 구현을 따로 해주지 않아도 된다.
    * DataLoader()를 사용하면 된다.
    * 물론, 이미지가 아닌 음성이나 다른 형식의 데이터를 사용한다면, DataLoader를 상속받아 새로 구현해주어야 한다.
    


### 2. Keras

* 모델을 Model() 혹은 Sequential()이라는 객체를 통해 구현할 수 있다.
    * 보통 Sequential() 객체에 layer를 쌓은 후 이 모델을 반환하는 함수를 많이 쓰는 것 같다.
    * 하지만 여기서는 Model을 그냥 전역변수를 만들어서 쭉 코딩하는 형태를 취했다.
    * layer를 쌓은 뒤에는 컴파일을 해주어야 한다. 컴파일할 때 optimizer와 loss를 명시한다.
    
    
* 학습 시 train_on_batch() 메소드를 사용하면 된다.
    * 이 외에도 .predict() / .fit() 등 다른 ML 라이브러리(TF / sklearn)들의 객체와 유사하다.
    * trainable이라는 멤버변수를 통해 모델의 일부를 학습시키지 않을 수도 있다.
        * Generator를 학습시키는 과정에서 사용되었다.
        
        
* Discriminator는 Discriminaotor 단독적으로 학습시키고, Generator는 얘를 직접 학습시키는 게 아니라 상위 모델인 GAN을 학습시키는 형태다.
    * 위에서 말했듯, 이 부분이 좀 헷갈렸다. 
    
    
* Pytorch를 먼저 쓰고 Keras를 쓰니깐 괜히 맘에 안드는 부분이 있는 것 같다.
    * 하지만 뭐가 됐든 TensorFlow보단 편한 것 같다.

---

# Epilogue

아무래도 원래 있던 코드를 내가 임의로 변경하다보니 결과물이 이상하게 나온 것 같다.

우선 g_loss가 수렴하지 않는다. 학습이 진행됨에 따라 점차 증가해야할 d_loss는 감소 수렴한다.

<img src="https://scontent-hkg3-2.xx.fbcdn.net/v/t1.0-9/20431352_1545641782167326_1401401417222284106_n.jpg?oh=e8f656a23c1ca509214782c5aabe53f4&oe=5A2F489C"></img>

<br>
또한, 결과물로 나오는 fake 이미지가 모두 1만 표현한다. 
<br>

<img src="https://scontent-hkg3-2.xx.fbcdn.net/v/t1.0-9/20429929_1545641018834069_1261116936006292570_n.jpg?oh=612bc7fdd7f9b2a8da89541457cd8555&oe=5A05AADE"></img>

조만간 이 문제사항에 대해 살펴본 후, ver2를 업로드할 예정이다.